<a href="https://colab.research.google.com/github/kridtapon/PortfolioOptimization-with-vectorbt/blob/main/PortfolioOptimization_with_vectorbt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

In [51]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from vectorbt.generic.nb import nanmean_nb
from vectorbt.portfolio.nb import order_nb, sort_call_seq_nb
from vectorbt.portfolio.enums import SizeType, Direction

In [52]:
data = yf.download("META AMZN NFLX GOOG AAPL TSLA INTC NVDA MSFT ORCL",start = '2019-01-01',end='2024-12-26',period='1d') #TSLA INTC NVDA MSFT ORCL

[*********************100%***********************]  10 of 10 completed


In [53]:
price = data['Close']

In [54]:
returns = price.pct_change()

In [55]:
# Plot normalized price series
(price / price.iloc[0]).vbt.plot().show()

In [56]:
print(returns.mean())

Ticker
AAPL    0.001438
AMZN    0.000956
GOOG    0.001077
INTC   -0.000211
META    0.001360
MSFT    0.001144
NFLX    0.001225
NVDA    0.003004
ORCL    0.001075
TSLA    0.002889
dtype: float64


In [57]:
print(returns.std())

Ticker
AAPL    0.019449
AMZN    0.021519
GOOG    0.019649
INTC    0.025976
META    0.026811
MSFT    0.018288
NFLX    0.027676
NVDA    0.032716
ORCL    0.019555
TSLA    0.040626
dtype: float64


In [58]:
print(returns.corr())

Ticker      AAPL      AMZN      GOOG      INTC      META      MSFT      NFLX  \
Ticker                                                                         
AAPL    1.000000  0.590207  0.637193  0.496123  0.553178  0.733963  0.450445   
AMZN    0.590207  1.000000  0.646812  0.432713  0.609516  0.681820  0.537590   
GOOG    0.637193  0.646812  1.000000  0.455290  0.634753  0.732493  0.449842   
INTC    0.496123  0.432713  0.455290  1.000000  0.400189  0.534619  0.342938   
META    0.553178  0.609516  0.634753  0.400189  1.000000  0.613306  0.485595   
MSFT    0.733963  0.681820  0.732493  0.534619  0.613306  1.000000  0.483529   
NFLX    0.450445  0.537590  0.449842  0.342938  0.485595  0.483529  1.000000   
NVDA    0.603316  0.577894  0.577211  0.493501  0.526486  0.671136  0.468565   
ORCL    0.470099  0.393313  0.434940  0.412574  0.354769  0.562097  0.305789   
TSLA    0.465463  0.420214  0.384550  0.337227  0.323073  0.431077  0.355637   

Ticker      NVDA      ORCL      TSLA  


In [59]:
np.random.seed(42)
symbols = price.columns  # Define the symbols list
# Generate random weights, n times
weights = []
for i in range(num_tests):
    w = np.random.random_sample(len(symbols))
    w = w / np.sum(w)
    weights.append(w)

print(len(weights))

2000


In [60]:
# Build column hierarchy such that one weight corresponds to one price series
_price = price.vbt.tile(num_tests, keys=pd.Index(np.arange(num_tests), name='symbol_group'))
_price = _price.vbt.stack_index(pd.Index(np.concatenate(weights), name='weights'))

print(_price.columns)

MultiIndex([( 0.07200801115525138,    0, 'AAPL'),
            ( 0.18278161119856334,    0, 'AMZN'),
            ( 0.14073105997227742,    0, 'GOOG'),
            ( 0.11509636655456425,    0, 'INTC'),
            (0.029995697219246602,    0, 'META'),
            (0.029991059956664158,    0, 'MSFT'),
            ( 0.01116698901526622,    0, 'NFLX'),
            ( 0.16652854641932946,    0, 'NVDA'),
            ( 0.11556865150895665,    0, 'ORCL'),
            ( 0.13613200699988054,    0, 'TSLA'),
            ...
            ( 0.12329327667570818, 1999, 'AAPL'),
            ( 0.16293601679565473, 1999, 'AMZN'),
            (   0.019208321944439, 1999, 'GOOG'),
            (  0.1057161266900053, 1999, 'INTC'),
            ( 0.19334527449896174, 1999, 'META'),
            (  0.1881953711245603, 1999, 'MSFT'),
            ( 0.01004536147411785, 1999, 'NFLX'),
            ( 0.06516776367083016, 1999, 'NVDA'),
            ( 0.09512782035300009, 1999, 'ORCL'),
            ( 0.03696466677272241,

In [61]:
# Define order size
size = np.full_like(_price, np.nan)
size[0, :] = np.concatenate(weights)  # allocate at first timestamp, do nothing afterwards

print(size.shape)

(1506, 20000)


In [62]:
# Run simulation
pf = vbt.Portfolio.from_orders(
    close=_price,
    size=size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True
) # all weights sum to 1, no shorting, and 100% investment in risky assets

print(len(pf.orders))

20000


In [75]:
# Plot annualized return against volatility, color by sharpe ratio
annualized_return = pf.annualized_return(freq='D')
annualized_return.index = pf.annualized_volatility(freq='D')
annualized_return.vbt.scatterplot(
    trace_kwargs=dict(
        mode='markers',
        marker=dict(
            color=pf.sharpe_ratio(freq='D'),
            colorbar=dict(
                title='sharpe_ratio'
            ),
            size=5,
            opacity=0.7
        )
    ),
    xaxis_title='annualized_volatility',
    yaxis_title='annualized_return'
).show()

In [76]:
# Get index of the best group according to the target metric
best_symbol_group = pf.sharpe_ratio(freq='D').idxmax()

print(best_symbol_group)

842


In [77]:
# Print best weights
print(weights[best_symbol_group])

[0.18770876 0.08326079 0.07486367 0.00378353 0.0427857  0.10152048
 0.03836667 0.32361831 0.10131189 0.04278019]


In [78]:
# Compute default stats
print(pf.iloc[best_symbol_group].stats())

Start                         2019-01-02 00:00:00
End                           2024-12-24 00:00:00
Period                                       1506
Start Value                                 100.0
End Value                             1719.168057
Total Return [%]                      1619.168057
Benchmark Return [%]                   833.591428
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        51.421147
Max Drawdown Duration                       374.0
Total Trades                                   10
Total Closed Trades                             0
Total Open Trades                              10
Open Trade PnL                        1619.168057
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



**Rebalancing Monthly**

In [79]:
# Select the first index of each month
rb_mask = ~_price.index.to_period('m').duplicated()

print(rb_mask.sum())

72


<ipython-input-79-cfbfc6a9a25c>:2: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.



In [80]:
rb_size = np.full_like(_price, np.nan)
rb_size[rb_mask, :] = np.concatenate(weights)  # allocate at mask

print(rb_size.shape)

(1506, 20000)


In [81]:
# Run simulation, with rebalancing monthly
rb_pf = vbt.Portfolio.from_orders(
    close=_price,
    size=rb_size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True,
    call_seq='auto'  # important: sell before buy
)

print(len(rb_pf.orders))

1439991


In [82]:
rb_best_symbol_group = rb_pf.sharpe_ratio(freq='D').idxmax()

print(rb_best_symbol_group)

993


In [84]:
print(weights[rb_best_symbol_group])

[0.18956998 0.02372812 0.05237607 0.00071884 0.06989136 0.02500977
 0.10887337 0.15592679 0.20137833 0.17252737]


In [85]:
print(rb_pf.iloc[rb_best_symbol_group].stats())

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2024-12-24 00:00:00
Period                                       1506
Start Value                                 100.0
End Value                             1130.962984
Total Return [%]                      1030.962984
Benchmark Return [%]                   833.591428
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        43.904311
Max Drawdown Duration                       387.0
Total Trades                                  319
Total Closed Trades                           309
Total Open Trades                              10
Open Trade PnL                         591.903547
Win Rate [%]                            88.349515
Best Trade [%]                         866.602081
Worst Trade [%]                        -48.913116
Avg Winning Trade [%]                   80.910881
Avg Losing Trade [%]                   -18.167899


In [86]:
def plot_allocation(rb_pf):
    # Plot weights development of the portfolio
    rb_asset_value = rb_pf.asset_value(group_by=False)
    rb_value = rb_pf.value()
    rb_idxs = np.flatnonzero((rb_pf.asset_flow() != 0).any(axis=1))
    rb_dates = rb_pf.wrapper.index[rb_idxs]
    fig = (rb_asset_value.vbt / rb_value).vbt.plot(
        trace_names=symbols,
        trace_kwargs=dict(
            stackgroup='one'
        )
    )
    for rb_date in rb_dates:
        fig.add_shape(
            dict(
                xref='x',
                yref='paper',
                x0=rb_date,
                x1=rb_date,
                y0=0,
                y1=1,
                line_color=fig.layout.template.layout.plot_bgcolor
            )
        )
    fig.show()

In [87]:
plot_allocation(rb_pf.iloc[rb_best_symbol_group])  # best group

**Search and rebalance every 30 days**

In [89]:
from numba import njit

In [90]:
srb_sharpe = np.full(price.shape[0], np.nan)

@njit
def pre_sim_func_nb(c, every_nth):
    # Define rebalancing days
    c.segment_mask[:, :] = False
    c.segment_mask[every_nth::every_nth, :] = True
    return ()

@njit
def find_weights_nb(c, price, num_tests):
    # Find optimal weights based on best Sharpe ratio
    returns = (price[1:] - price[:-1]) / price[:-1]
    returns = returns[1:, :]  # cannot compute np.cov with NaN
    mean = nanmean_nb(returns)
    cov = np.cov(returns, rowvar=False)  # masked arrays not supported by Numba (yet)
    best_sharpe_ratio = -np.inf
    weights = np.full(c.group_len, np.nan, dtype=np.float64)

    for i in range(num_tests):
        # Generate weights
        w = np.random.random_sample(c.group_len)
        w = w / np.sum(w)

        # Compute annualized mean, covariance, and Sharpe ratio
        p_return = np.sum(mean * w) * ann_factor
        p_std = np.sqrt(np.dot(w.T, np.dot(cov, w))) * np.sqrt(ann_factor)
        sharpe_ratio = p_return / p_std
        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            weights = w

    return best_sharpe_ratio, weights

@njit
def pre_segment_func_nb(c, find_weights_nb, history_len, ann_factor, num_tests, srb_sharpe):
    if history_len == -1:
        # Look back at the entire time period
        close = c.close[:c.i, c.from_col:c.to_col]
    else:
        # Look back at a fixed time period
        if c.i - history_len <= 0:
            return (np.full(c.group_len, np.nan),)  # insufficient data
        close = c.close[c.i - history_len:c.i, c.from_col:c.to_col]

    # Find optimal weights
    best_sharpe_ratio, weights = find_weights_nb(c, close, num_tests)
    srb_sharpe[c.i] = best_sharpe_ratio

    # Update valuation price and reorder orders
    size_type = SizeType.TargetPercent
    direction = Direction.LongOnly
    order_value_out = np.empty(c.group_len, dtype=np.float64)
    for k in range(c.group_len):
        col = c.from_col + k
        c.last_val_price[col] = c.close[c.i, col]
    sort_call_seq_nb(c, weights, size_type, direction, order_value_out)

    return (weights,)

@njit
def order_func_nb(c, weights):
    col_i = c.call_seq_now[c.call_idx]
    return order_nb(
        weights[col_i],
        c.close[c.i, c.col],
        size_type=SizeType.TargetPercent
    )

In [93]:
vbt.settings.array_wrapper['freq'] = 'D'

In [95]:
ann_factor = returns.vbt.returns.ann_factor

In [96]:
# Run simulation using a custom order function
srb_pf = vbt.Portfolio.from_order_func(
    price,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(find_weights_nb, -1, ann_factor, num_tests, srb_sharpe),
    cash_sharing=True,
    group_by=True
)

In [97]:
# Plot best Sharpe ratio at each rebalancing day
pd.Series(srb_sharpe, index=price.index).vbt.scatterplot(trace_kwargs=dict(mode='markers')).show()

In [98]:
print(srb_pf.stats())

Start                                 2019-01-02 00:00:00
End                                   2024-12-24 00:00:00
Period                                 1506 days 00:00:00
Start Value                                         100.0
End Value                                      772.158158
Total Return [%]                               672.158158
Benchmark Return [%]                           833.591428
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                56.140824
Max Drawdown Duration                   577 days 00:00:00
Total Trades                                          262
Total Closed Trades                                   252
Total Open Trades                                      10
Open Trade PnL                                 220.254835
Win Rate [%]                                    70.634921
Best Trade [%]                                 384.934092
Worst Trade [%

In [99]:
plot_allocation(srb_pf)

In [100]:
# Run simulation, but now consider only the last 252 days of data
srb252_sharpe = np.full(price.shape[0], np.nan)

srb252_pf = vbt.Portfolio.from_order_func(
    price,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(find_weights_nb, 252, ann_factor, num_tests, srb252_sharpe),
    cash_sharing=True,
    group_by=True
)

In [101]:
pd.Series(srb252_sharpe, index=price.index).vbt.scatterplot(trace_kwargs=dict(mode='markers')).show()

In [102]:
print(srb252_pf.stats())

Start                                 2019-01-02 00:00:00
End                                   2024-12-24 00:00:00
Period                                 1506 days 00:00:00
Start Value                                         100.0
End Value                                      514.804795
Total Return [%]                               414.804795
Benchmark Return [%]                           833.591428
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                53.273967
Max Drawdown Duration                   555 days 00:00:00
Total Trades                                          207
Total Closed Trades                                   197
Total Open Trades                                      10
Open Trade PnL                                  93.092449
Win Rate [%]                                    66.497462
Best Trade [%]                                    284.277
Worst Trade [%

In [103]:
plot_allocation(srb252_pf)